In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [3]:
# Set dataset Path
with_mask_dir='/kaggle/input/face-mask-dataset/data/with_mask/'
without_mask_dir="/kaggle/input/face-mask-dataset/data/without_mask/"


In [4]:
def load_data():
    data=[]
    labels =[]
    for category in ["with_mask","without_mask"]:
        folder = os.path.join("/kaggle/input/face-mask-dataset/data",category)
        label = 0 if category == "without_mask" else 1
        for file in os.listdir(folder):
            img_path = os.path.join(folder,file)
            image = tf.keras.preprocessing.image.load_img(img_path,target_size=(128,128))
            image = tf.keras.preprocessing.image.img_to_array(image)
            image = tf.keras.applications.mobilenet.preprocess_input(image)
            data.append(image)
            labels.append(label)
            
    return np.array(data),np.array(labels)
            
        

In [5]:
X,Y = load_data()

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [6]:
X.shape

(7553, 128, 128, 3)

In [7]:
Y.shape

(7553,)

In [8]:
#Splitting the datasets
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=7)

In [9]:
X_train.shape

(6042, 128, 128, 3)

In [10]:
X_test.shape

(1511, 128, 128, 3)

In [18]:
data_gen = ImageDataGenerator(
rotation_range=20,
zoom_range=0.15,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.15,
horizontal_flip=True,
fill_mode="nearest"
)

In [12]:
model = Sequential(
[
  Conv2D(32,(3,3),padding="same",activation="relu",input_shape=(128,128,3)),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),
    
Conv2D(64,(3,3),padding="same",activation="relu"),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.3),
    
Flatten(),
Dense(128,activation="relu"),
    Dropout(0.4),
    Dense(1,activation="sigmoid")
    
    
])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(optimizer=Adam(learning_rate=0.001),loss="binary_crossentropy",metrics=["accuracy"])


In [19]:
history = model.fit(
data_gen.flow(X_train,y_train,batch_size=32),
validation_data=(X_test,y_test),
epochs=30)

Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1728111273.147719     729 service.cc:145] XLA service 0x7d1c20003c40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728111273.147782     729 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1728111273.147787     729 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  1/189 ━━━━━━━━━━━━━━━━━━━━ 37:46 12s/step - accuracy: 0.7188 - loss: 0.6990

I0000 00:00:1728111282.316264     729 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


189/189 ━━━━━━━━━━━━━━━━━━━━ 49s 199ms/step - accuracy: 0.7345 - loss: 0.9181 - val_accuracy: 0.8769 - val_loss: 0.3630
Epoch 2/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 30s 154ms/step - accuracy: 0.8366 - loss: 0.3902 - val_accuracy: 0.8915 - val_loss: 0.3098
Epoch 3/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 30s 155ms/step - accuracy: 0.8454 - loss: 0.3679 - val_accuracy: 0.8915 - val_loss: 0.2808
Epoch 4/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 30s 155ms/step - accuracy: 0.8464 - loss: 0.3454 - val_accuracy: 0.9047 - val_loss: 0.2315
Epoch 5/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 31s 156ms/step - accuracy: 0.8664 - loss: 0.3072 - val_accuracy: 0.9272 - val_loss: 0.2004
Epoch 6/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 31s 157ms/step - accuracy: 0.8772 - loss: 0.2902 - val_accuracy: 0.9113 - val_loss: 0.2458
Epoch 7/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 30s 155ms/step - accuracy: 0.8818 - loss: 0.2782 - val_accuracy: 0.9239 - val_loss: 0.1835
Epoch 8/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 30s 154ms/step - accuracy: 0.8848 - loss: 0.2618 - val

In [20]:
model.save("model.h5")

In [21]:
model.evaluate(X_test,y_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9676 - loss: 0.1017


[0.09524877369403839, 0.9682329297065735]

## Creating Model with Hyperparameter tuning

In [16]:
def build_model(hp):
    model = Sequential([
        
       Conv2D(
        filters=hp.Int("conv_1_filter",min_value=20,max_value=130,step=10),
        kernel_size=hp.Choice("conv_1_kernel",values=[2,5]),
        activation="relu",
        input_shape=(128,128,3)),
        MaxPooling2D(pool_size=hp.Choice("max_pooling_1",values=[2,5])),
        Dropout(hp.Choice("dropout_1",values=[1e-1,1e-2,1e-3,1e-4])),
        
        Conv2D(
        filters=hp.Int("conv_2_filter",min_value=10,max_value=120,step=10),
        kernel_size=hp.Choice("conv_2_kernel",values=[2,5]),
        activation="relu"),
        MaxPooling2D(pool_size=hp.Choice("max_pooling_2",values=[2,5])),
        Dropout(hp.Choice("dropout_2",values=[1e-1,1e-2,1e-3,1e-4])),
        
        
        Conv2D(
        filters=hp.Int("conv_3_filter",min_value=10,max_value=120,step=10),
        kernel_size=hp.Choice("conv_3_kernel",values=[2,5]),
        activation="relu"),
        MaxPooling2D(pool_size=hp.Choice("max_pooling_3",values=[2,5])),
        Dropout(hp.Choice("dropout_3",values=[1e-1,1e-2,1e-3,1e-4])),
        
        Flatten(),
        Dense(
        units=hp.Int("dense_1",min_value=10,max_value=200,step=10),
        activation="relu"),
        Dense(1,activation="sigmoid")        
    ])
    
    model.compile(optimizer=Adam(
    hp.Choice("learning_rate",values=[1e-1,1e-2,1e-3,1e-4])),
                 loss="binary_crossentropy",
                 metrics=["accuracy"])
    return model

In [31]:
tuner = RandomSearch(
build_model,
objective="val_accuracy",
max_trials=10,
directory="tuner_weights",
project_name="face Mask Detection")

Reloading Tuner from tuner_weights/face Mask Detection/tuner0.json


In [33]:
tuner.search(X_train,y_train,validation_data=(X_test,y_test),epochs = 4)

Trial 10 Complete [00h 00m 37s]
val_accuracy: 0.9053606986999512

Best val_accuracy So Far: 0.9252150654792786
Total elapsed time: 00h 10m 34s


In [46]:
model3 = tuner.get_best_models(num_models=1)[0]

In [47]:
model3.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 124, 124, 30)   │         2,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 30)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 30)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 23, 23, 100)    │        12,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 11, 11, 100)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 11, 11, 100)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 10, 60)     │        24,060 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 2, 2, 60)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 2, 2, 60)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 240)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │        48,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 86,841 (339.22 KB)

 Trainable params: 86,841 (339.22 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model3.fit(X_train,y_train,epochs=100,batch_size=30)

Epoch 1/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - accuracy: 0.8930 - loss: 0.2688
Epoch 2/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8964 - loss: 0.2453
Epoch 3/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9226 - loss: 0.1995
Epoch 4/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9244 - loss: 0.1916
Epoch 5/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9225 - loss: 0.1888
Epoch 6/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9299 - loss: 0.1997
Epoch 7/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9324 - loss: 0.1760
Epoch 8/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9339 - loss: 0.1732
Epoch 9/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9411 - loss: 0.1554
Epoch 10/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9487 - loss: 0.1435
Epoch 11/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9437 - loss: 0.1497
Epoch 12/100
202/202 ━━━━━━━━━━━━━━━━━━

In [49]:
model3.evaluate(X_test,y_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9657 - loss: 0.1612


[0.15919874608516693, 0.9616148471832275]

In [52]:
model3.save("predicting_model.h5")